In [1]:
import xarray as xr
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys

path = '/home/akazemi3/Desktop/MB_Lab_Project/'
sys.path.append(path)
from analysis.neural_data_regression.tools.regression import *

from tools.processing import *
from tools.loading import *

import pandas as pd
import seaborn as sns
import pickle
from analysis.neural_data_regression.tools.utils import get_activations_iden, get_scores_iden
BEST_ALPHA_PATH = '/data/atlas/regression_alphas'    
ROOT_PATH = '/data/atlas/model_scores'

In [2]:
# def get_activations_iden(dataset, model_info):
    
#         model_name = model_info['iden'] 
        
#         if model_info['max_pool']:
#             model_name = model_name + '_mp' 

#         activations_identifier = model_name + '_' + f'{model_info["num_layers"]}_layers' + '_' + f'{model_info["num_features"]}_features' 

#         if model_info['pca']:
#             activations_identifier = activations_identifier + '_' + f'{model_info["pca_type"]}' + '_' + f'{model_info["max_pca_components"]}' + '_'  + f'{model_info["pca_dataset"]}' + '_' + 'pcs' 

#         if mode == 'cv' and dataset == 'naturalscenes':
#             return activations_identifier + '_' + dataset + '_' + 'shared'
  
#         else:
#             return activations_identifier + '_' + dataset 



# def get_scores_iden(model_info, activations_identifier, region, mode, alpha):
    
#     if model_info['pca']:
#         activations_identifier = activations_identifier.split(f'_{dataset}')[0] 
#         scores_identifier = activations_identifier + '_' + f'{model_info["num_pca_components"]}' + '_' + 'subset' + '_' + dataset + '_' + region + '_' + mode + '_' + f'ridge(alpha={alpha})'
#     else:
#         scores_identifier = activations_identifier + '_' + region + '_' + mode + '_' + f'ridge(alpha={alpha})' 
#     return scores_identifier



In [3]:
def get_best_alpha(data_dict, dataset, mode, region):
    
            
    root_path = '/data/atlas/model_scores'
    df = pd.DataFrame()

    index = 0
    for model_name, model_info in data_dict.items():
                
        for alpha in model_info['alphas']:

            regression = f'Ridge(alpha={alpha})' 
                

            activations_iden = get_activations_iden(model_info, dataset, mode)
            scores_iden = get_scores_iden(model_info, activations_iden, region, dataset, mode, alpha)
            
            data = xr.open_dataset(os.path.join(root_path,activations_iden,scores_iden))
            r_values = data.where(data.region == region,drop=True).r_value.values
            mean_r = np.mean(r_values)
            df_tmp =  pd.DataFrame({'mean_score':mean_r,
                                    'alpha':alpha,
                                    'model':model_name},index=[index])
            df = pd.concat([df,df_tmp])
            index += 1

    
    df = df.groupby(['model','alpha']).mean().reset_index()
    max_idx = df.groupby(['model']).agg({'mean_score':('idxmax')}).reset_index()['mean_score'].tolist()
    df_max_alphas = df.loc[max_idx].reset_index(drop=True)

    return df_max_alphas
    

# Untrained alexnet

### pca

In [7]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_u_wide_pca'
MAX_POOL = False    

for region in REGIONS:
    data_dict = {

    
        'alexnet u wide 256 pcs':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetUPCA(n_components=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':10000,
                'pca_dataset':'nsd',
                'n_dims':256,
                'max_dims':5000,
                'dim_reduction_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]}, 

        'alexnet u wide 1000 pcs':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetUPCA(n_components=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':10000,
                'pca_dataset':'nsd',
                'n_dims':1000,
                'max_dims':5000,
                'dim_reduction_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},     
        
         
        'alexnet u wide 5000 pcs':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetUPCA(n_components=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':10000,
                'pca_dataset':'nsd',
                'n_dims':5000,
                'max_dims':5000,
                'dim_reduction_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},
}


    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
                     model  alpha  mean_score
0  alexnet u wide 1000 pcs    100    0.212800
1   alexnet u wide 256 pcs    100    0.210379
2  alexnet u wide 5000 pcs    100    0.213531


### random projection

In [5]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_u_wide_rp'


MAX_POOL = False                
for region in REGIONS:
    data_dict = {
        
        'alexnet u wide 256 rps':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetU(filters_5 = 10000, num_projections=256, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':10000,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'n_dims':256,
                'alphas': [10**i for i in range(1,4)]},  

    
        
    
       'alexnet u wide 1000 rps':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetU(filters_5 = 10000, num_projections=1000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':10000,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'n_dims':1000,
                'alphas': [10**i for i in range(1,4)]},  
        
        
    
           'alexnet u wide 5000 rps':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetU(filters_5 = 10000, num_projections=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':10000,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'n_dims':5000,
                'alphas': [10**i for i in range(1,4)]},       
    }
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
                     model  alpha  mean_score
0  alexnet u wide 1000 rps     10    0.190630
1   alexnet u wide 256 rps     10    0.168940
2  alexnet u wide 5000 rps     10    0.206167


### filters

In [9]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_u_wide_mp'


MAX_POOL = True

for region in REGIONS:
    data_dict = {
        
       'alexnet u wide 10 filters':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetUWide(filters_5 = 10, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':10,
                'dim_reduction_type':None,
                 'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},  
        
    
         
       'alexnet u wide 100 filters':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetUWide(filters_5 = 100, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':100,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},     
    
    
       'alexnet u wide 1000 filters':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetUWide(filters_5 = 1000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':1000,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},  

    
    
       'alexnet u wide 10000 filters':{
                'iden':'alexnet_u_wide',
                #'model':AlexnetUWide(filters_5 = 10000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':10000,
                'dim_reduction_type':None,
                 'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},   
            }
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
                          model  alpha  mean_score
0     alexnet u wide 10 filters     10    0.084169
1    alexnet u wide 100 filters     10    0.118336
2   alexnet u wide 1000 filters     10    0.161645
3  alexnet u wide 10000 filters     10    0.188697


# Engineered Model

### filters

In [8]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'model_abs_3x3_bp_224_mp'


MAX_POOL = True

for region in REGIONS:
    data_dict = {
        'model abs 3x3 bp 224 10 filters':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBP(filters_3 = 10, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]},  

    
    
        'model abs 3x3 bp 224 100 filters':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBP(filters_3 = 100, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':100,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]},  
        
    
         
        'model abs 3x3 bp 224 1000 filters':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBP(filters_3 = 1000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':1000,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]},    


        'model abs 3x3 bp 224 10000 filters':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBP(filters_3 = 10000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10000,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]},  
    }
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
                                model  alpha  mean_score
0     model abs 3x3 bp 224 10 filters    100    0.080929
1    model abs 3x3 bp 224 100 filters    100    0.133710
2   model abs 3x3 bp 224 1000 filters    100    0.176358
3  model abs 3x3 bp 224 10000 filters    100    0.201933


### pca 

In [8]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'model_abs_3x3_bp_224_pca'


MAX_POOL = False                
for region in REGIONS:
    data_dict = {

        'model abs 3x3 bp 224 256 pcs':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBPPCA(filters_3 = 10000, n_components=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10000,
                'pca':True,
                'pca_dataset':'nsd',
                'num_pca_components':256,
                'max_pca_components':5000,
                'pca_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]}, 

        'model abs 3x3 bp 224 1000 pcs':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBPPCA(filters_3 = 10000, n_components=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10000,
                'pca':True,
                'pca_dataset':'nsd',
                'num_pca_components':1000,
                'max_pca_components':5000,
                'pca_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,5)]},   
        
         
        'model abs 3x3 bp 224 5000 pcs':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBPPCA(filters_3 = 10000, n_components=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10000,
                'pca':True,
                'pca_dataset':'nsd',
                'num_pca_components':5000,
                'max_pca_components':5000,
                'pca_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]},     
    }

    df_best_alpha = get_best_alpha(data_dict)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
                           model  alpha  mean_score
0  model abs 3x3 bp 224 1000 pcs    100    0.215848
1   model abs 3x3 bp 224 256 pcs    100    0.212651
2  model abs 3x3 bp 224 5000 pcs    100    0.216922


### random projections

In [4]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'model_abs_3x3_bp_224_rp'

MAX_POOL = False              


for region in REGIONS:
    data_dict = {        
        'model abs 3x3 bp 224 256 rps':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBP(filters_3 = 10000, num_projections=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10000,
                'n_dims':256,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'max_dims':256,
                'alphas': [10**i for i in range(2,5)]},  

        'model abs 3x3 bp 224 1000 rps':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBP(filters_3 = 10000, num_projections=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10000,
                'n_dims':1000,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'max_dims':1000,
                'alphas': [10**i for i in range(2,5)]},  
        
         
        'model abs 3x3 bp 224 5000 rps':{
                'iden':'model_abs_3x3_bp_224',
                #'model':EngModel3LAbsBP(filters_3 = 10000, num_projections=5000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10000,
                'n_dims':5000,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'max_dims':5000,
                'alphas': [10**i for i in range(2,5)]},     
   }

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
                           model  alpha  mean_score
0  model abs 3x3 bp 224 1000 rps    100    0.192254
1   model abs 3x3 bp 224 256 rps    100    0.168394
2  model abs 3x3 bp 224 5000 rps    100    0.208484


# Alexnet

### filters

In [11]:
    
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_mp'


MAX_POOL = True                
for region in REGIONS:
    data_dict = {
        
       'alexnet':{
                'iden':'alexnet',
                #'model':Alexnet(global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(3,7)]},            
        
    }
    
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()
    

V4
     model   alpha  mean_score
0  alexnet  100000    0.208461


In [17]:
    
dataset = 'naturalscenes'
REGIONS = ['V1','V2','V3','V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_u_mp'


MAX_POOL = True                
for region in REGIONS:
    data_dict = {
        
    
       'alexnet u conv1':{
                'iden':'alexnet_u_conv1',
                #'model':AlexnetU(features_layer =2, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':1,
                'num_features':64,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,6)]},   
       
        'alexnet u conv2':{
                'iden':'alexnet_u_conv2',
                #'model':AlexnetU(features_layer =5, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':2,
                'num_features':192,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},   
       
        'alexnet u conv3':{
                'iden':'alexnet_u_conv3',
                #'model':AlexnetU(features_layer =7, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':3,
                'num_features':384,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},   
       
        'alexnet u conv4':{
                'iden':'alexnet_u_conv4',
                #'model':AlexnetU(features_layer =9, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':4,
                'num_features':256,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},       
        
        'alexnet u':{
                'iden':'alexnet_u',
                #'model':AlexnetU(global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]}, 
    }
    
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()
    

V1
             model  alpha  mean_score
0        alexnet u     10    0.228549
1  alexnet u conv1   1000    0.157694
2  alexnet u conv2    100    0.219965
3  alexnet u conv3     10    0.252143
4  alexnet u conv4     10    0.256974
V2
             model  alpha  mean_score
0        alexnet u     10    0.187537
1  alexnet u conv1   1000    0.125786
2  alexnet u conv2    100    0.181930
3  alexnet u conv3    100    0.208063
4  alexnet u conv4     10    0.215074
V3
             model  alpha  mean_score
0        alexnet u     10    0.154471
1  alexnet u conv1   1000    0.107891
2  alexnet u conv2    100    0.150582
3  alexnet u conv3    100    0.172681
4  alexnet u conv4     10    0.177915
V4
             model  alpha  mean_score
0        alexnet u     10    0.136087
1  alexnet u conv1   1000    0.111485
2  alexnet u conv2    100    0.140064
3  alexnet u conv3    100    0.154744
4  alexnet u conv4     10    0.156733


In [18]:
    
dataset = 'naturalscenes'
REGIONS = ['V1','V2','V3','V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_mp'


MAX_POOL = True                
for region in REGIONS:
    data_dict = {
        
    
       'alexnet conv1':{
                'iden':'alexnet_conv1',
                #'model':Alexnet(features_layer =2, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':1,
                'num_features':64,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(3,7)]},            
    
       'alexnet conv2':{
                'iden':'alexnet_conv2',
                #'model':Alexnet(features_layer =5, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':2,
                'num_features':192,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(3,7)]},      
    
       'alexnet conv3':{
                'iden':'alexnet_conv3',
                #'model':Alexnet(features_layer =7, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':3,
                'num_features':384,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(3,9)]},    
    
       'alexnet conv4':{
                'iden':'alexnet_conv4',
               #'model':Alexnet(features_layer =9, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':4,
                'num_features':256,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(3,7)]},     
        
       'alexnet':{
                'iden':'alexnet',
               #'model':Alexnet(global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(3,7)]},      
    }
    
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()
    

V1
           model    alpha  mean_score
0        alexnet   100000    0.239833
1  alexnet conv1    10000    0.168182
2  alexnet conv2   100000    0.246923
3  alexnet conv3  1000000    0.265467
4  alexnet conv4   100000    0.248129
V2
           model    alpha  mean_score
0        alexnet   100000    0.215266
1  alexnet conv1    10000    0.135811
2  alexnet conv2   100000    0.211548
3  alexnet conv3  1000000    0.235125
4  alexnet conv4   100000    0.222569
V3
           model    alpha  mean_score
0        alexnet   100000    0.197312
1  alexnet conv1    10000    0.118892
2  alexnet conv2   100000    0.181828
3  alexnet conv3  1000000    0.208914
4  alexnet conv4   100000    0.201077
V4
           model    alpha  mean_score
0        alexnet   100000    0.208461
1  alexnet conv1   100000    0.126201
2  alexnet conv2   100000    0.178070
3  alexnet conv3  1000000    0.207005
4  alexnet conv4   100000    0.205107


### pca

In [6]:
    
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_mp_pca'


MAX_POOL = True                
for region in REGIONS:
    data_dict = {

    'alexnet 10 pcs':{
                'iden':'alexnet',
                #'model':AlexnetPCA(n_components=256, global_mp = MAX_POOL).Build(),
                'layers': ['pca5'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'pca':True,
                'pca_dataset':'nsd',
                'num_pca_components':10,
                'max_pca_components':256,
                'pca_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]}, 
        
        'alexnet 100 pcs':{
                'iden':'alexnet',
                #'model':AlexnetPCA(n_components=256, global_mp = MAX_POOL).Build(),
                'layers': ['pca5'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'pca':True,
                'pca_dataset':'nsd',
                'num_pca_components':100,
                'max_pca_components':256,
                'pca_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]}, 
        
        'alexnet 256 pcs':{
                'iden':'alexnet',
                #'model':AlexnetPCA(n_components=256, global_mp = MAX_POOL).Build(),
                'layers': ['pca5'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'pca':True,
                'pca_dataset':'nsd',
                'num_pca_components':256,
                'max_pca_components':256,
                'pca_type':'pca',
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(2,5)]}, 
        
        
#         'alexnet 256 pcs':{
#                 'iden':'alexnet',
#                 #'model':AlexnetPCA(n_components=5000, global_mp = MAX_POOL).Build(),
#                 'layers': ['pca5'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':256,
#                 'pca':True,
#                 'pca_dataset':'nsd',
#                 'num_pca_components':256,
#                 'max_pca_components':5000,
#                 'pca_type':'pca',
#                 'max_pool':MAX_POOL,
#                 'alphas': [10**i for i in range(2,5)]},
        
                 # 'alexnet 1000 pcs':{
#                 'iden':'alexnet',
#                 #'model':AlexnetPCA(n_components=5000, global_mp = MAX_POOL).Build(),
#                 'layers': ['pca5'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':256,
#                 'pca':True,
#                 'pca_dataset':'nsd',
#                 'num_pca_components':1000,
#                 'max_pca_components':5000,
#                 'pca_type':'pca',
#                 'max_pool':MAX_POOL,
#                 'alphas': [10**i for i in range(2,5)]}, 
    
#         'alexnet 5000 pcs':{
#                 'iden':'alexnet',
#                 #'model':AlexnetPCA(n_components=5000, global_mp = MAX_POOL).Build(),
#                 'layers': ['pca5'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':256,
#                 'pca':True,
#                 'pca_dataset':'nsd',
#                 'num_pca_components':5000,
#                 'max_pca_components':5000,
#                 'pca_type':'pca',
#                 'max_pool':MAX_POOL,
#                 'alphas': [10**i for i in range(2,5)]}, 

        }

df_best_alpha = get_best_alpha(data_dict)
print(region)
print(df_best_alpha)
dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
pickle.dump(dict_best_alpha, file)
file.close()
    
    
    


V4
             model  alpha  mean_score
0   alexnet 10 pcs  10000    0.193673
1  alexnet 100 pcs  10000    0.202346
2  alexnet 256 pcs  10000    0.193292


### random projections

In [5]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_mp_rp'
MAX_POOL = True               


for region in REGIONS:
    data_dict = {
        
        'alexnet 10 rps':{
                'iden':'alexnet',
                #'model':Alexnet(num_projections=256, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'n_dims':10,
                'alphas': [10**i for i in range(3,7)]},  


        'alexnet 100 rps':{
                'iden':'alexnet',
                #'model':Alexnet(num_projections=256, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'n_dims':100,
                'alphas': [10**i for i in range(3,7)]},  


        'alexnet 256 rps':{
                'iden':'alexnet',
                #'model':Alexnet(num_projections=256, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'dim_reduction_type':'rp',
                'max_pool':MAX_POOL,
                'n_dims':256,
                'alphas': [10**i for i in range(3,7)]},  

    
    
#         'alexnet 1000 rps':{
#                 'iden':'alexnet',
#                 #'model':Alexnet(num_projections=1000, global_mp = MAX_POOL).Build(),
#                 'layers': ['last'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':256,
#                 'dim_reduction_type':'rp',
#                 'max_pool':MAX_POOL,
#                 'n_dims':1000,
#                 'alphas': [10**i for i in range(3,7)]},    
        
    
         
#         'alexnet 5000 rps':{
#                 'iden':'alexnet',
#                 #'model':Alexnet(num_projections=5000, global_mp = MAX_POOL).Build(),
#                 'layers': ['last'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':256,
#                 'dim_reduction_type':'rp',
#                 'max_pool':MAX_POOL,
#                 'n_dims':5000,
#                 'alphas': [10**i for i in range(3,7)]}, 
    }
    
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
             model   alpha  mean_score
0   alexnet 10 rps   10000    0.129820
1  alexnet 100 rps  100000    0.199531
2  alexnet 256 rps  100000    0.208461


# Alexnet Untrained

### filters

In [9]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'alexnet_u_mp'
MAX_POOL = True               


for region in REGIONS:
    data_dict = {
        'alexnet u':{
                'iden':'alexnet_u',
                #'model':AlexnetU(global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessRGB, 
                'num_layers':5,
                'num_features':256,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,4)]},   
    
    
    
    }
    
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
       model  alpha  mean_score
0  alexnet u     10    0.136087


# Avg Pool Model

In [7]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
#alpha_file_name = 'model_3L_mp'
alpha_file_name = 'model_abs_3x3_bp_224_ap_mp'
MAX_POOL = True               


for region in REGIONS:
    data_dict = {

    
    'model abs 3x3 bp 224 ap 10 filters':{
                'iden':'model_abs_3x3_bp_224_ap',
                #'model':EngModel3LAbsBPAP(filters_3 = 10, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,5)]},  
    
    'model abs 3x3 bp 224 ap 100 filters':{
                'iden':'model_abs_3x3_bp_224_ap',
                #'model':EngModel3LAbsBPAP(filters_3 = 100, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':100,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,5)]},  
    
    'model abs 3x3 bp 224 ap 1000 filters':{
                'iden':'model_abs_3x3_bp_224_ap',
                #'model':EngModel3LAbsBPAP(filters_3 = 1000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':1000,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,5)]},  
    
    'model abs 3x3 bp 224 ap 10000 filters':{
                'iden':'model_abs_3x3_bp_224_ap',
                #'model':EngModel3LAbsBPAP(filters_3 = 10000, global_mp = MAX_POOL).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=224).PreprocessGS, 
                'num_layers':3,
                'num_features':10000,
                'dim_reduction_type':None,
                'max_pool':MAX_POOL,
                'alphas': [10**i for i in range(1,5)]},  

   
#        'alexnet u wide 10 filters':{
#                 'iden':'alexnet_u_wide',
#                 #'model':AlexnetUWide(filters_5 = 10, global_mp = MAX_POOL).Build(),
#                 'layers': ['last'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':10,
#                 'dim_reduction_type':None,
#                  'max_pool':MAX_POOL,
#                 'alphas': [10**i for i in range(1,4)]},  
        
    
         
#        'alexnet u wide 100 filters':{
#                 'iden':'alexnet_u_wide',
#                 #'model':AlexnetUWide(filters_5 = 100, global_mp = MAX_POOL).Build(),
#                 'layers': ['last'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':100,
#                 'dim_reduction_type':None,
#                 'max_pool':MAX_POOL,
#                 'alphas': [10**i for i in range(1,4)]},     
    
    
#        'alexnet u wide 1000 filters':{
#                 'iden':'alexnet_u_wide',
#                 #'model':AlexnetUWide(filters_5 = 1000, global_mp = MAX_POOL).Build(),
#                 'layers': ['last'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':1000,
#                 'dim_reduction_type':None,
#                 'max_pool':MAX_POOL,
#                 'alphas': [10**i for i in range(1,4)]},  

    
    
#        'alexnet u wide 10000 filters':{
#                 'iden':'alexnet_u_wide',
#                 #'model':AlexnetUWide(filters_5 = 10000, global_mp = MAX_POOL).Build(),
#                 'layers': ['last'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':10000,
#                 'dim_reduction_type':None,
#                  'max_pool':MAX_POOL,
#                 'alphas': [10**i for i in range(1,4)]},   
        
    
#        'alexnet':{
#                 'iden':'alexnet',
#                 #'model':Alexnet(global_mp = MAX_POOL).Build(),
#                 'layers': ['last'], 
#                 'preprocess':Preprocess(im_size=224).PreprocessRGB, 
#                 'num_layers':5,
#                 'num_features':256,
#                 'dim_reduction_type':None,
#                 'max_pool':MAX_POOL,
#                 'alphas': [10**i for i in range(3,7)]},           
    
    
       # 'alexnet u':{
       #          'iden':'alexnet_u',
       #          #'model':AlexnetU(global_mp = MAX_POOL).Build(),
       #          'layers': ['last'], 
       #          'preprocess':Preprocess(im_size=224).PreprocessRGB, 
       #          'num_layers':5,
       #          'num_features':256,
       #          'dim_reduction_type':None,
       #          'max_pool':MAX_POOL,
       #          'alphas': [10**i for i in range(1,4)]},  
    }
    
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
                                   model  alpha  mean_score
0     model abs 3x3 bp 224 ap 10 filters     10    0.092791
1    model abs 3x3 bp 224 ap 100 filters     10    0.151616
2   model abs 3x3 bp 224 ap 1000 filters     10    0.189704
3  model abs 3x3 bp 224 ap 10000 filters     10    0.212995


In [18]:
dataset = 'naturalscenes'
REGIONS = ['V4']
# dataset = 'majajhong'
# REGIONS = ['V4','IT']
mode = 'train'
alpha_file_name = 'scat_transform_kymatio_J3_L4_rgb'
MAX_POOL = True               


for region in REGIONS:
    data_dict = {
        
        

    'scattering transform kymatio':{
                'iden':'scat_transform_kymatio_J3_L4_rgb',
                #'model':ScatTransformKymatio(J = 3, L = 8, M = 32, N = 32, flatten = True, global_mp= False).Build(),
                'layers': ['last'], 
                'preprocess':Preprocess(im_size=32).PreprocessGS, 
                'num_layers':2,
                'num_features':'x',
                'dim_reduction_type':None,
                'max_pool':False,
                'alphas': [0] + [10**i for i in range(3)]},  
    

    }
    
    

    df_best_alpha = get_best_alpha(data_dict, dataset, mode, region)
    print(region)
    print(df_best_alpha)
    dict_best_alpha = df_best_alpha.set_index(['model']).to_dict()['alpha']
    file = open(os.path.join(BEST_ALPHA_PATH, f'{alpha_file_name}_{dataset}_{region}'),'wb')
    pickle.dump(dict_best_alpha, file)
    file.close()

V4
                          model  alpha  mean_score
0  scattering transform kymatio     10    0.191269
